In [5]:
import numpy as np 
import pulp
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# Paramètres
N = 40  # Nombre de points de collecte (sans compter le dépôt)
K = 4  # Nombre de camions
Q = 15  # Capacité des camions en tonnes
M = 1000  # Grande constante pour les contraintes de temps



In [14]:
# Demandes de collecte pour chaque point (y compris le dépôt qui est 0)
demandes = [0, 2, 4, 3, 5, 2, 3, 4, 1, 5, 2, 1, 5, 3, 3, 2, 3, 1, 2, 5, 2, 2, 1, 3, 5, 2, 5, 4, 1, 1, 2,2, 4, 3, 2, 2, 3, 1, 9, 5, 2]  # en tonnes
early_times = [
    0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 
    15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 
    25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 
    35, 36, 37, 38, 39, 40, 41, 42, 43,44
]
# print(len(early_times))
late_times = [
    10, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 
    25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 
    35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 
    45, 46, 47, 48, 49, 50, 51, 52, 53,54
]
# print(len(late_times))
service_times = [0, 1, 1, 1, 1, 1, 1,1,1,1,1, 1, 1, 1, 1, 1, 1,1,1,1,1, 1, 1, 1, 1, 1, 1,1,1,1,1, 1, 1, 1, 1, 1, 1,1,1,1,1] # temps de collecte en heures

len(service_times)

41

In [15]:
import numpy as np

def generate_distance_matrix(num_points):
    # Initialize a random distance matrix
    np.random.seed(42)  # for reproducibility
    distance_matrix = np.random.randint(5, 15, size=(num_points, num_points))
    
    # Make the matrix symmetric (distance from i to j is the same as from j to i)
    for i in range(num_points):
        for j in range(i, num_points):
            distance_matrix[i][j] = distance_matrix[j][i]
    
    # Set the diagonal elements to 0 (distance from a point to itself is 0)
    np.fill_diagonal(distance_matrix, 0)
    
    return distance_matrix

# Example usage
num_points = 40
distances = generate_distance_matrix(num_points)

# Print the generated distance matrix
print(distances)


[[ 0 13 11 ... 11 11 13]
 [13  0 13 ...  6  5  5]
 [11 13  0 ...  8 10  9]
 ...
 [11  6  8 ...  0 12 13]
 [11  5 10 ... 12  0 12]
 [13  5  9 ... 13 12  0]]


In [16]:
# Modèle de minimisation de coût
model = pulp.LpProblem("Optimisation_Collecte_Dechets", pulp.LpMinimize)


In [17]:
# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(N+1) for j in range(N+1) for k in range(K)), cat='Binary')
t = pulp.LpVariable.dicts("t", (i for i in range(N+1)), lowBound=0, cat='Continuous')


In [18]:
# Fonction objectif : minimiser la distance totale parcourue
model += pulp.lpSum(distances[i][j] * x[i, j, k] for i in range(N) for j in range(N) if i != j for k in range(K))


In [20]:
# Contraintes

# Chaque point de collecte est visité exactement une fois
for j in range(1, N):
    model += pulp.lpSum(x[i, j, k] for i in range(N) if i != j for k in range(K)) == 1

# Contrainte de capacité pour chaque camion
for k in range(K):
    model += pulp.lpSum(demandes[i] * x[i, j, k] for i in range(N) for j in range(1, N) if i != j) <= Q

# Contrainte de fenêtre de temps
for i in range(1, N):
    model += early_times[i] <= t[i]
    model += t[i] <= late_times[i]

# Temps d'arrivée doit respecter les temps de parcours
for i in range(N):
    for j in range(1, N):
        if i != j:
            for k in range(K):
                model += t[j] >= t[i] + service_times[i] + distances[i][j] - M * (1 - x[i, j, k])


In [22]:
# Résoudre le modèle
model.solve()

# Display the results
print("Status:", pulp.LpStatus[model.status])
print("Coût total (distance):", pulp.value(model.objective))

for k in range(K):
    print(f"\nCamion {k + 1} :")
    for i in range(N):
        for j in range(N):
            if i != j and pulp.value(x[i, j, k]) == 1:
                print(f"De {i} à {j} avec coût {distances[i][j]} et temps d'arrivée {pulp.value(t[j]):.2f}")


Status: Optimal
Coût total (distance): 232.0

Camion 1 :
De 0 à 3 avec coût 14 et temps d'arrivée 14.00
De 0 à 13 avec coût 8 et temps d'arrivée 17.00
De 0 à 20 avec coût 5 et temps d'arrivée 24.00
De 0 à 22 avec coût 5 et temps d'arrivée 26.00
De 0 à 25 avec coût 5 et temps d'arrivée 29.00
De 0 à 33 avec coût 5 et temps d'arrivée 37.00
De 1 à 19 avec coût 5 et temps d'arrivée 23.00
De 1 à 24 avec coût 5 et temps d'arrivée 28.00
De 2 à 11 avec coût 5 et temps d'arrivée 25.00
De 10 à 17 avec coût 5 et temps d'arrivée 31.00
De 10 à 31 avec coût 5 et temps d'arrivée 35.00
De 20 à 28 avec coût 5 et temps d'arrivée 32.00
De 22 à 30 avec coût 5 et temps d'arrivée 44.00

Camion 2 :
De 1 à 5 avec coût 5 et temps d'arrivée 19.00
De 1 à 10 avec coût 5 et temps d'arrivée 19.00
De 6 à 8 avec coût 5 et temps d'arrivée 22.00
De 8 à 37 avec coût 5 et temps d'arrivée 41.00
De 10 à 26 avec coût 5 et temps d'arrivée 40.00
De 11 à 35 avec coût 5 et temps d'arrivée 39.00
De 13 à 23 avec coût 5 et temps d'

In [23]:
import random

depot = 0  # Dépôt

# Paramètres du Recuit Simulé
initial_temperature = 1000
cooling_rate = 0.95
stopping_temperature = 1e-8
max_iterations = 1000

# Fonction de coût
def calculate_cost(solution):
    total_cost = 0
    for route in solution:
        route_cost = 0
        for i in range(len(route) - 1):
            route_cost += distances[route[i]][route[i + 1]]
        total_cost += route_cost
    return total_cost

# Fonction pour générer une solution initiale
def generate_initial_solution():
    solution = []
    nodes = list(range(1, N + 1))
    for _ in range(K):
        route = [depot]
        load = 0
        while nodes and load + demandes[nodes[0]] <= Q:
            node = nodes.pop(0)
            route.append(node)
            load += demandes[node]
        route.append(depot)
        solution.append(route)
    return solution

# Fonction pour générer une solution voisine
def generate_neighbor(solution):
    neighbor = [route[:] for route in solution]
    i, j = random.randint(0, K - 1), random.randint(1, len(neighbor[0]) - 2)
    if neighbor[i][j] != depot:
        neighbor[i][j], neighbor[(i + 1) % K][j] = neighbor[(i + 1) % K][j], neighbor[i][j]
    return neighbor

# Fonction pour exécuter le Recuit Simulé
def simulated_annealing():
    current_solution = generate_initial_solution()
    current_cost = calculate_cost(current_solution)
    best_solution = current_solution
    best_cost = current_cost
    temperature = initial_temperature

    while temperature > stopping_temperature:
        for _ in range(max_iterations):
            neighbor = generate_neighbor(current_solution)
            neighbor_cost = calculate_cost(neighbor)
            cost_diff = neighbor_cost - current_cost

            if cost_diff < 0 or np.exp(-cost_diff / temperature) > random.random():
                current_solution = neighbor
                current_cost = neighbor_cost
                if current_cost < best_cost:
                    best_solution = current_solution
                    best_cost = current_cost

        temperature *= cooling_rate
    return best_solution, best_cost

# Exécution de l'algorithme
best_solution, best_cost = simulated_annealing()
print("Meilleure solution:", best_solution)
print("Coût total:", best_cost)

Meilleure solution: [[0, 5, 11, 7, 13, 0], [0, 15, 2, 3, 4, 9, 0], [0, 10, 16, 17, 8, 14, 0], [0, 1, 6, 12, 18, 19, 20, 0]]
Coût total: 192
